In [1]:
import json
import os
import sys

## 1. Align Text

In [2]:
with open('/home/legg/wmt-2014/training/concat.fr-en.BPE_aligned.fr', 'r') as f:
    fr = f.read().split('<<<<<my split line>>>>>')[:-1]

with open('/home/legg/wmt-2014/training/concat.fr-en.BPE_aligned.en', 'r') as f:
    en = f.read().split('<<<<<my split line>>>>>')[:-1]

In [3]:
print(len(fr))
print(len(en))

5435126
5435126


In [169]:
with open('/home/legg/wmt-2014/dev/newstest2013.BPE_aligned.fr', 'r') as f:
    fr_dev = f.read().split('<<<<<my split line>>>>>')[:-1]

with open('/home/legg/wmt-2014/dev/newstest2013.BPE_aligned.en', 'r') as f:
    en_dev = f.read().split('<<<<<my split line>>>>>')[:-1]

In [170]:
print(len(fr_dev))
print(len(en_dev))

3000
3000


## 2. Trim and merge Vocab

In [6]:
with open('/home/legg/wmt-2014/training/concat.fr-en.BPE.fr.json', 'r') as f:
    fr_vocab = json.load(f)

with open('/home/legg/wmt-2014/training/concat.fr-en.BPE.en.json', 'r') as f:
    en_vocab = json.load(f)

In [7]:
print(len(fr_vocab))
print(len(en_vocab))

31267
26498


In [4]:
# total vocab size
print(len(set(fr_vocab.keys()) | set(en_vocab.keys())))

36002


In [5]:
fr_vocab

{'<EOS>': 0,
 '<GO>': 1,
 '<UNK>': 2,
 'de': 3,
 'la': 4,
 'et': 5,
 'le': 6,
 'les': 7,
 'à': 8,
 'des': 9,
 'en': 10,
 'que': 11,
 'du': 12,
 'pour': 13,
 'dans': 14,
 'un': 15,
 'une': 16,
 'est': 17,
 'qui': 18,
 'sur': 19,
 'nous': 20,
 'au': 21,
 'a': 22,
 'par': 23,
 'ce': 24,
 'vous': 25,
 'il': 26,
 'pas': 27,
 'plus': 28,
 'ne': 29,
 'avec': 30,
 'ou': 31,
 'je': 32,
 'aux': 33,
 'sont': 34,
 'se': 35,
 "d'@@": 36,
 'cette': 37,
 "l'@@": 38,
 'd’@@': 39,
 '-': 40,
 'l’@@': 41,
 'mais': 42,
 'ces': 43,
 'ont': 44,
 ':': 45,
 '(@@': 46,
 'être': 47,
 'été': 48,
 'si': 49,
 'comme': 50,
 'son': 51,
 'votre': 52,
 'tout': 53,
 '.': 54,
 ',': 55,
 'notre': 56,
 'leur': 57,
 'ses': 58,
 'commission': 59,
 'y': 60,
 'fait': 61,
 'tous': 62,
 'on': 63,
 "d'un": 64,
 "d'une": 65,
 'aussi': 66,
 'bien': 67,
 'pays': 68,
 'é': 69,
 'elle': 70,
 'faire': 71,
 'très': 72,
 'nos': 73,
 'sa': 74,
 'également': 75,
 'même': 76,
 "c'est": 77,
 'peut': 78,
 'entre': 79,
 'ant': 80,
 'sans': 81

In [20]:
fr_inv_vocab = {v:k for k,v in fr_vocab.items()}
en_inv_vocab = {v:k for k,v in en_vocab.items()}

### Manually trim non-latin characters

In [21]:
en_inv_vocab[26498-2752]

'▸'

In [22]:
26498-2752

23746

In [23]:
fr_inv_vocab[31267-2199]

'名@@'

In [24]:
31267-2199

29068

In [25]:
fr_vocab_pp = {v:int(k) for k,v in fr_inv_vocab.items() if int(k) < 29068 }
fr_inv_vocab_pp = {int(v):k for k,v in fr_vocab_pp.items()}

In [26]:
fr_inv_vocab_pp[29067]

'Ú'

In [27]:
fr_vocab_pp['Ú']

29067

In [28]:
en_vocab_pp = {v:k for k,v in en_inv_vocab.items() if int(k) < 23746}
en_inv_vocab_pp = {v:k for k,v in en_vocab_pp.items()}

In [29]:
en_inv_vocab_pp[23745]

'ė'

In [30]:
en_vocab_pp['ė']

23745

In [31]:
# total vocab size
print(len(set(fr_vocab_pp.keys()) | set(en_vocab_pp.keys())))

32981


In [32]:
last_idx_fr = len(fr_inv_vocab_pp)
last_idx_en = len(en_inv_vocab_pp)
fr_inv_vocab_pp[last_idx_fr] = '<PAD>'
en_inv_vocab_pp[last_idx_en] = '<PAD>'
fr_vocab_pp['<PAD>'] = last_idx_fr
en_vocab_pp['<PAD>'] = last_idx_en

In [33]:
with open('/home/legg/wmt-2014/training/pp_vocab.en', 'w') as f:
    json.dump(en_vocab_pp, f)
    
with open('/home/legg/wmt-2014/training/pp_vocab.fr', 'w') as f:
    json.dump(fr_vocab_pp, f)
    
with open('/home/legg/wmt-2014/training/pp_inv_vocab.en', 'w') as f:
    json.dump(en_inv_vocab_pp, f)
    
with open('/home/legg/wmt-2014/training/pp_inv_vocab.fr', 'w') as f:
    json.dump(fr_inv_vocab_pp, f)

In [34]:
import json
with open('/home/legg/wmt-2014/training/pp_inv_vocab.en', 'r') as f:
    dum = json.load(f)

In [35]:
dum['0']

'<EOS>'

## 3. Preprocess corpus

In [201]:
def pp_corpus(corpus_lines, word2idx_lookup):
    symbol_corpus = []
    for ln in corpus_lines:
        tokens = ln.rstrip('\n').split(' ')
        symbols = [word2idx_lookup.get(tok, word2idx_lookup['<UNK>']) for tok in tokens]
        symbol_corpus.append(symbols)
    return symbol_corpus

In [202]:
en_pp = pp_corpus(corpus_lines=en, word2idx_lookup=en_vocab_pp)
fr_pp = pp_corpus(corpus_lines=fr, word2idx_lookup=fr_vocab_pp)

In [203]:
en_pp[:5]

[[2427, 39, 408, 1706, 16, 3107, 13, 502, 1428, 2838],
 [2427, 39, 408, 7115, 6842, 16, 3107, 13, 502, 1428, 2838],
 [233, 4, 1041, 3704, 4, 935, 162],
 [29, 8817, 56, 15, 1147, 14740],
 [1706, 5, 233, 7032, 11480, 131, 3, 4710, 7734, 2599, 222, 9, 24, 7315, 9094]]

In [204]:
with open('/home/legg/wmt-2014/training/concat.fr-en.BPE_pp.en', 'w') as f:
    json.dump(en_pp, f)
    
with open('/home/legg/wmt-2014/training/concat.fr-en.BPE_pp.fr', 'w') as f:
    json.dump(fr_pp, f)

In [205]:
en_dev_pp = pp_corpus(corpus_lines=en_dev, word2idx_lookup=en_vocab_pp)
fr_dev_pp = pp_corpus(corpus_lines=fr_dev, word2idx_lookup=fr_vocab_pp)

In [206]:
with open('/home/legg/wmt-2014/dev/newstest2013.BPE_pp.en', 'w') as f:
    json.dump(en_dev_pp, f)
    
with open('/home/legg/wmt-2014/dev/newstest2013.BPE_pp.fr', 'w') as f:
    json.dump(fr_dev_pp, f)

In [207]:
en_inv_vocab_pp[2]

'<UNK>'